<a href="https://colab.research.google.com/github/kero2no1/data_analytics/blob/working/Data_analytics_outcome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>